**Required Libraries**





FAISS for vector database,
Beautifulsoup for web scraping,
Gradio for UI,
LLM Model = BLOOM,
SMTP for email


In [ ]:
pip install faiss-cpu

In [ ]:
import os
import json
from transformers import pipeline
from bs4 import BeautifulSoup
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import requests
import sqlite3
import faiss
import numpy as np

**Using BLOOM Model**

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoModelForCausalLM, AutoTokenizer

model_name = "bigscience/bloom-560m"
# BLOOM Model
model = AutoModelForCausalLM.from_pretrained(model_name,device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)

**Chat Buffer initialization**

In [ ]:
#chat buffer memory initialization
chat_buffer = []

# generator initialization
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Response based on the user's input
def generate_response(input_text):
    # Generate a response using the BLOOM model
    response = generator(input_text, max_length=200,truncation=True)
    return response[0]['generated_text']

In [ ]:
#Extract information from the internet using web scraping
def extract_info(url):
    # Send an HTTP request to the URL
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the text from the HTML content
    # text = soup.get_text()
    text = ' '.join([p.get_text() for p in soup.find_all('p')])

    return text

In [ ]:
extract_info("https://www.analyticsvidhya.com/blog/2024/10/function-calling-in-ai-agents-using-mistral-7b/")

'Mastering Python’s Set Difference: A Game-Changer for Data Wrangling Function calling in large language models (LLMs) has transformed how AI agents interact with external systems, APIs, or tools, enabling structured decision-making based on natural language prompts. By using JSON schema-defined functions, these models can autonomously select and execute external operations, offering new levels of automation. This article will demonstrate how function calling can be implemented using Mistral 7B, a state-of-the-art model designed for instruction-following tasks. This article was published as a part of the\xa0Data Science Blogathon. In the scope of Generative AI (GenAI), AI agents represent a significant evolution in artificial intelligence capabilities. These agents use models, such as large language models (LLMs), to create content, simulate interactions, and perform complex tasks autonomously. The AI agents enhance their functionality and applicability across various domains, includin

**Send Emails**

In [ ]:
# Initialize the email server
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()

# Define a function to email the specified personnel based on the recent interaction
def send_email(subject, body, recipient):
    # Construct the email message
    msg = MIMEMultipart()
    msg['From'] = os.getenv('SENDER EMAIL')
    msg['To'] = recipient
    msg['Subject'] = subject

    # Attach the body of the email
    msg.attach(MIMEText(body, 'plain'))

    # Send the email
    server.login(msg['From'], os.getenv('EMAIL PASSWORD'))
    text = msg.as_string()
    server.sendmail(msg['From'], msg['To'], text)

**Store scraped web pages in a local repository**

In [ ]:
def store_scraped_page(url, text):
    # Connect to the SQLite database
    conn = sqlite3.connect('/content/scraped_pages.db')
    c = conn.cursor()

    # Creating the table if it doesn't exist
    c.execute('''CREATE TABLE IF NOT EXISTS scraped_pages
                 (url text, text text)''')

    # Inserting the scraped page into the table
    c.execute("INSERT INTO scraped_pages VALUES (?, ?)", (url, text))

    # Commiting the changes and close the connection
    conn.commit()
    conn.close()

In [ ]:
store_scraped_page("https://www.analyticsvidhya.com/blog/2024/10/function-calling-in-ai-agents-using-mistral-7b/","test")

**Function to search the local repository for a specific query**

In [ ]:
def search_local_repository(query):
    # Connect to the SQLite database
    conn = sqlite3.connect('/content/scraped_pages.db')
    c = conn.cursor()

    # Search the table for the query
    c.execute("SELECT text FROM scraped_pages WHERE text LIKE ?", ('%' + query + '%',))

    # Fetch the results
    results = c.fetchall()

    # Close the connection
    conn.close()

    return results

In [ ]:
search_local_repository("test")

[('test',), ('test',)]

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_name)

# function to build a vector database
def build_vector_database():
    # Connecting to the SQLite database
    conn = sqlite3.connect('/content/scraped_pages.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS scraped_pages
                 (url text, text text)''')
    # Fetching all the scraped pages
    c.execute("SELECT text FROM scraped_pages")
    results = c.fetchall()
    # Creating a vector database
    vectors = []
    for result in results:
        # Converting the result to a string before creating the vector
        text = str(result[0])
        # Checking if the text is not empty before creating the vector
        if text:
            # vector = np.array(text)
            # vectors.append(vector)
            vector = model.encode(text)
            vectors.append(vector)
    if vectors:
      # Converting list of vectors into a NumPy array
      vectors_np = np.array(vectors)
      # FAISS index creation
      index = faiss.IndexFlatL2(vectors_np.shape[1])  # Use the size of the vector's dimension
      index.add(vectors_np)  # Add vectors to the FAISS index
      faiss.write_index(index, "vector_database.index")  # Save the index
      print("Vector database built and saved to 'vector_database.index'.")
    else:
      print("No vectors found to build the index.")

    # Check if vectors is empty before proceeding
    """if vectors and vectors[0].size > 0:
        index = faiss.IndexFlatL2(vectors[0].size)  # Use .size for NumPy arrays
        index.add(np.array(vectors))
        faiss.write_index(index, "vector_database.index")
    else:
        print("No vectors found to build the index.")"""


In [ ]:
build_vector_database()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Vector database built and saved to 'vector_database.index'.


**Query the vector database**

In [ ]:
def query_vector_database(query):
    # Loading the index
    index = faiss.read_index("vector_database.index")
    # Create a query vector
    # query_vector = np.array(query)
    query_vector = model.encode(query) #Encoding the query using the sentence transformer model
    query_vector = query_vector.astype('float32') #Converting to float32
    k=5
    # Search the index
    D, I = index.search(np.array([query_vector]),k)
    # Return the results
    return D, I

In [ ]:
query_vector_database("sample query text")

(array([[4.5036289e+04, 4.5036289e+04, 2.3746169e+05, 3.4028235e+38,
         3.4028235e+38]], dtype=float32),
 array([[ 0,  2,  1, -1, -1]]))

**Chat Agent Main**

In [ ]:
# Define the chat agent's user interface
def chat_agent():
    print("Welcome to the chat agent!")
    while True:
        # Get the user's input
        user_input = input("User: ")
        # Checking if the user wants to query the vector database
        if "query" in user_input.lower():
            # Querying the vector database
            D, I = query_vector_database(user_input)
            # Printing the results
            for i in range(len(D[0])):
                print("Result {}: {}".format(i+1, D[0][i]))
        # Check if the user wants to extract information from the internet
        elif "extract info" in user_input.lower():
            # Get the URL from the user
            url = input("Enter the URL: ")
            # Extract the information from the internet
            text = extract_info(url)
            # Print the extracted information
            print("Extracted Info: ", text)
            # Store the scraped page in the local repository
            store_scraped_page(url, text)
        # Check if the user wants to email the specified personnel
        elif "send email" in user_input.lower():
            # Get the subject, body, and recipient from the user
            subject = input("Enter the subject: ")
            body = input("Enter the body: ")
            recipient = input("Enter the recipient's email: ")
            # Send the email
            send_email(subject, body, recipient)
        # Generate a response based on the user's input
        else:
            response = generate_response(user_input)

            # Print the response
            print("Chat Agent: ", response)

            # Add the user's input and the response to the chat buffer
            chat_buffer.append(user_input)
            chat_buffer.append(response)

# Run the chat agent
# chat_agent()

In [ ]:
!pip install -q gradio


In [ ]:
import gradio as gr
demo = gr.ChatInterface(
    fn=chat_agent,
    title="Chat Agent",
    description="A simple chat agent that can query a vector database, extract information from the internet, and send emails.",
)

# Launch the Gradio interface
demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:228: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:980: UserWarning: Expected 0 arguments for function <function chat_agent at 0x79cecc32a440>, received 2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:988: UserWarning: Expected maximum 0 arguments for function <function chat_agent at 0x79cecc32a440>, received 2.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bdb13297227386b3b3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2014, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1565, in call_function
    prediction = await fn(*processed_input)
  File "/usr/local/lib/python3.10/dist-packages/gradio/utils.py", line 813, in async_wrapper
    response = await f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py", line 638, in _submit_fn
    response = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_async

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7e123bdc4cd139d51c.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://86dcd9aff35fb8fbe6.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://2aa11afeaed163070b.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://ad7e93e70d2ffad0ff.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://bdb13297227386b3b3.gradio.live
